# Brain Volume Analysis

In [1]:
import graspologic as gp
import hyppo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.spatial.distance import squareform
from statsmodels.stats.multitest import multipletests

from pkg.data import (
    GENOTYPES,
    HEMISPHERES,
    SUPER_STRUCTURES,
    load_vertex_metadata,
    load_volume,
)

In [2]:
## Read the data
data, labels = load_volume()
vertex_name, vertex_hemispheres, vertex_structures = load_vertex_metadata()

print(data.shape)
print(labels.shape)

(29, 332)
(29,)


## Testing whether group volumes are significantly different

We compute 3-sample distance correlation to see if there is any differences in the 3 groups. Specifically, we test whether the distribution of brain volumes from at least two genotypes are different.

In [3]:
ksample = hyppo.ksample.KSample("dcorr")

brain_volumes = [data[labels == genotype] for genotype in GENOTYPES]

stat, pval = ksample.test(*brain_volumes)

print(pval)

0.006007509501547217


We see significant difference among the three genotypes

## Finding Signal Communities

We stratify by hemisphere, super structure, then hemisphere + super structure

### Testing for significant hemispheres

In [4]:
hemisphere_volumes = []

for hemisphere in HEMISPHERES:
    to_append = [
        brain_volumes[idx][:, vertex_hemispheres == hemisphere]
        for idx in range(len(GENOTYPES))
    ]
    hemisphere_volumes.append(to_append)

In [5]:
# Run Dcorr
stats, pvals = [], []

for idx in range(len(hemisphere_volumes)):
    to_test = hemisphere_volumes[idx]
    stat, pval = ksample.test(*to_test)

    stats.append(stat)
    pvals.append(pval)

significant, corrected_pvals, _, corrected_alpha = multipletests(pvals, method="holm")

print(corrected_pvals)

[0.01730104 0.00929847]


### Testing for significant super structures

In [6]:
structure_volumes = []

for structure in SUPER_STRUCTURES:
    to_append = [
        brain_volumes[idx][:, vertex_structures == structure]
        for idx in range(len(GENOTYPES))
    ]
    structure_volumes.append(to_append)

In [7]:
# Run Dcorr
stats, pvals = [], []

for idx in range(len(structure_volumes)):
    to_test = structure_volumes[idx]
    stat, pval = ksample.test(*to_test)

    stats.append(stat)
    pvals.append(pval)

significant, corrected_pvals, _, corrected_alpha = multipletests(pvals, method="holm")

print(corrected_pvals)

[0.04506916 0.01793374 0.01793374 0.04506916 0.83478065]


### Testing for significant regions

In [10]:
from tqdm.notebook import tqdm, trange

In [ ]:
# Run Dcorr
stats, pvals = [], []

ksample = hyppo.ksample.KSample("dcorr")

for idx in trange(vertex_name.size):
    to_test = [brain_volumes[jdx][:, idx : idx + 1] for jdx in range(len(GENOTYPES))]
    stat, pval = ksample.test(
        *to_test, reps=100000, workers=-1, auto=False
    )

    stats.append(stat)
    pvals.append(pval)

significant, corrected_pvals, _, corrected_alpha = multipletests(pvals, method="holm")

  0%|          | 0/332 [00:00<?, ?it/s]

In [75]:
print(corrected_alpha)

0.00015060240963855423


In [ ]:
sort_idx = np.argsort(corrected_pvals)

tmp = np.array(
[
    vertex_name[sort_idx],
    vertex_hemispheres[sort_idx],
    vertex_structures[sort_idx],
    corrected_pvals[sort_idx],
    np.array(pvals)[sort_idx]
]
)

res = pd.DataFrame(tmp.T, columns = ["Name", "Hemisphere", "Structure", "Corrected Pval", "Pval"])

In [ ]:
res.to_csv("../results/outputs/volume_analysis.csv", index=False)

In [ ]:
res.head()

In [77]:
res["Corrected Pval"] <= corrected_alpha

0      False
1      False
2      False
3      False
4      False
       ...  
327    False
328    False
329    False
330    False
331    False
Name: Corrected Pval, Length: 332, dtype: bool